In [2]:
import numpy as np
from my_ueg import my_ueg
from pyscf import lib, gto, scf, cc

In [3]:
nocc = 7 # 7, 19, 27, 33, 57, 81, 93
my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc), gamma=np.sqrt(2))
h0 = my_sys.madelung() / 2
h1 = my_sys.get_h1_real()
cderi = my_sys.get_cderi_real()

mol = gto.M()
mol.nelectron = my_sys.nparticle
mol.nao = my_sys.nkpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

dm = np.zeros((my_sys.nkpts,my_sys.nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol).density_fit()
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(my_sys.nkpts)
mf._cderi = cderi
mf.init_guess = "1e"
mf.max_cycle = -1
escf = mf.kernel(dm0=dm)

mf.mo_coeff = np.eye(my_sys.nkpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/my_sys.nparticle}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
mycc.conv_tol = 1e-7
mycc.kernel()

et = mycc.ccsd_t()
print("CCSD energy", mycc.e_corr)
print("CCSD energy per electron", mycc.e_corr / my_sys.nparticle)
print("CCSD(T) energy", et)
print("CCSD(T) energy per electron", et / my_sys.nparticle)



******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmp_9h9kv3_
max_memory 20000 MB (current use 126 MB)
Set gradient conv threshold to 3.16228e-05
init E= 8.49148146743777
  HOMO = 1.04145818327265  LUMO = 2.32324526529525
SCF not converged.
SCF energy = 8.49148146743777
escf: 8.491481467437769
escf per electron: 0.6065343905312692

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 7, nmo = 19
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 127 MB)
Init t2, MP2 energy = 8.11699308199585  E_corr(MP2) -0.374488385441918
Init E_corr(RCCSD) = -0.374488385441919
cycle = 1  E_c

Overwritten attributes  get_hcore energy_nuc get_ovlp  of <class 'pyscf.df.df_jk.DFRHF'>


cycle = 2  E_corr(RCCSD) = -0.294796808642051  dE = -0.0610901803  norm(t1,t2) = 0.0390142
cycle = 3  E_corr(RCCSD) = -0.276436461619712  dE = 0.018360347  norm(t1,t2) = 0.0166986
cycle = 4  E_corr(RCCSD) = -0.276492710722161  dE = -5.62491024e-05  norm(t1,t2) = 7.13585e-05
cycle = 5  E_corr(RCCSD) = -0.276499060608661  dE = -6.3498865e-06  norm(t1,t2) = 7.17236e-06
cycle = 6  E_corr(RCCSD) = -0.276499450074825  dE = -3.89466164e-07  norm(t1,t2) = 3.75654e-07
cycle = 7  E_corr(RCCSD) = -0.276499419547778  dE = 3.05270471e-08  norm(t1,t2) = 6.18114e-08
RCCSD converged
E(RCCSD) = 8.214982047889992  E_corr = -0.2764994195477781
RCCSD(T) correction = -0.00143386619297451
CCSD energy -0.27649941954777807
CCSD energy per electron -0.019749958539127005
CCSD(T) energy -0.0014338661929745112
CCSD(T) energy per electron -0.00010241901378389366


In [4]:
my_sys.prep_afqmc(mycc)

# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis (k-points): 19
# Number of CholVecs (q-points): 92
